# Pipeline

**Loading of datasets**

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import zscore, iqr
from sklearn.model_selection import train_test_split
import numpy as np
import missingno as msno
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
data_weather = pd.read_csv("spanish-cities-energy-consumption/weather_features.csv")
data_generation = pd.read_csv("spanish-cities-energy-consumption/energy_dataset.csv")
data_weather['dt_iso'] = pd.to_datetime(data_weather['dt_iso'])
data_generation['time'] = pd.to_datetime(data_generation['time'])


**Data Exploration**

In [ ]:
data_weather.head()

In [ ]:
a= 35064*5
b= 178396 -a
print(a)
print(b)

In [ ]:
duplicates = data_weather[data_weather['dt_iso'].duplicated(keep=False)]  # keep=False to include the first occurrence as well

# Sort the duplicates based on 'dt_iso' in ascending order (default)
duplicates_sorted = duplicates[['dt_iso', 'city_name']].sort_values(by='dt_iso')

# Display the sorted duplicates
#print(duplicates_sorted)
print(pd.concat([duplicates_sorted.head(), duplicates_sorted.tail()]))

In [ ]:
data_weather.tail()

In [ ]:
data_generation.head()

In [ ]:
data_generation.tail()

In [ ]:
data_weather.describe()

In [ ]:
data_generation.describe()


In [10]:
data_generation.drop(columns=["generation fossil coal-derived gas",
                              "generation hydro pumped storage aggregated", 
                              "generation fossil oil shale",
                              "generation fossil peat", 
                              "generation geothermal", 
                              "generation marine", 
                              "generation wind offshore",
                              "forecast wind offshore eday ahead" ], inplace=True)

In [ ]:
for col, dtype in data_weather.dtypes.items():
    print(f"Column '{col}' has data type: {dtype}")

In [ ]:
for col, dtype in data_generation.dtypes.items():
    print(f"Column '{col}' has data type: {dtype}")

In [ ]:
print(data_weather.isnull().values.any())

In [ ]:
print(data_generation.isnull().values.any())

In [15]:
#Divide the numerical features from the categorical features
weather_num = data_weather.select_dtypes(include='number')
weather_num.drop('weather_id', axis=1, inplace=True)
weather_cat = data_weather.select_dtypes(exclude='number').drop('dt_iso', axis=1)
weather_cat['weather_id']=data_weather['weather_id']
generation_num = data_generation.select_dtypes(include='number')

In [ ]:
#see the unique values of categorical features to check for outliers
unique_values_all = {col: weather_cat[col].unique() for col in weather_cat.columns}

print(unique_values_all)


In [17]:
def visualData(data, time):
   for col in data.columns:
        print(col)
     # Plot Histogram of Z-scores
        plt.figure(figsize=(12, 5))

        # Histogram plot
        plt.subplot(1, 2, 1)
        plt.hist(data[col], bins=100, edgecolor='k', alpha=0.7)
        plt.title(f'Histogram of values {col}')
        plt.xlabel('value')
        plt.ylabel('Frequency')
        plt.legend()

        # Scatter Plot with Z-scores
        plt.subplot(1, 2, 2)
        plt.scatter(time, data[col], alpha=0.7)
        plt.title(f'Scatter Plot of values over time {col}')
        plt.xlabel('time')
        plt.ylabel('value')
        plt.grid(True)

        plt.tight_layout()
        plt.show()

In [ ]:
visualData(data=weather_num, time=data_weather['dt_iso'])

In [ ]:
visualData(data=generation_num, time=data_generation['time'])

In [20]:
#weather dataset
normal_weather_columns=["temp", 'temp_min', 'temp_max', 'pressure']
skewed_weather_columns=['humidity', 'wind_speed', 'wind_deg', 'rain_1h', 'rain_3h', 'snow_3h', 'clouds_all']

#energy dataset

normal_energy_columns=['generation biomass', 'generation fossil hard coal', 'generation fossil oil',
                        'generation other', 'generation other renewable', 'total load forecast','total load actual',
                         'price day ahead', 'price actual']
skewed_energy_columns=['generation fossil brown coal/lignite', 'generation fossil gas','generation hydro pumped storage consumption',
                        'generation hydro run-of-river and poundage', 'generation hydro water reservoir', 'generation nuclear',
                         'generation solar', 'generation waste', 'generation wind onshore',
                           'forecast solar day ahead', 'forecast wind onshore day ahead']

**Missing Values**

In [ ]:
missing_values_sum = data_generation.isnull().sum()
missing_values_sum

In [ ]:


columns_to_median_impute=['generation fossil brown coal/lignite', 'generation fossil gas','generation hydro pumped storage consumption',
                        'generation hydro run-of-river and poundage', 'generation hydro water reservoir', 'generation nuclear',
                         'generation solar', 'generation waste', 'generation wind onshore']

columns_to_mean_impute=['generation biomass', 'generation fossil hard coal', 'generation fossil oil',
                        'generation other', 'generation other renewable', 'total load actual']

generation_num[columns_to_median_impute] = generation_num[columns_to_median_impute].fillna(generation_num.median(numeric_only=True))
generation_num[columns_to_mean_impute] = generation_num[columns_to_mean_impute].fillna(generation_num.mean(numeric_only=True))


print(generation_num.isna().sum())

print("Number of missing values now is:", pd.isnull(generation_num).sum().sum())

**Outliers**

In [23]:
def Zscore(data):
# Calculate Z-scores for each data point
    z_scores = zscore(data)
    # Define a threshold for identifying outliers
    threshold = 3
    # Identify outliers
    outliers = (z_scores > threshold) | (z_scores < -threshold)
    return outliers, z_scores

weather_outliers, weather_zscores = Zscore(weather_num[normal_weather_columns])
generation_outliers, generation_zscore = Zscore(generation_num[normal_energy_columns])

In [24]:
def visualZscore(z_scores, outliers, threshold=3):
    for col in z_scores.columns:
        # Plot Histogram of Z-scores
        plt.figure(figsize=(12, 5))

        # Histogram plot
        plt.subplot(1, 2, 1)
        plt.hist(z_scores[col], bins=100, edgecolor='k', alpha=0.7)
        plt.axvline(threshold, color='r', linestyle='dashed', linewidth=1.5, label=f'Outlier Threshold (+{threshold})')
        plt.axvline(-threshold, color='r', linestyle='dashed', linewidth=1.5, label=f'Outlier Threshold (-{threshold})')
        plt.title(f'Histogram of Z-scores {col}')
        plt.xlabel('Z-score')
        plt.ylabel('Frequency')
        plt.yscale('log')
        plt.legend()

        # Scatter Plot with Z-scores
        plt.subplot(1, 2, 2)
        plt.scatter(range(len(z_scores[col])), z_scores[col], c=['red' if outlier else 'blue' for outlier in outliers[col]], alpha=0.7)
        plt.axhline(threshold, color='r', linestyle='dashed', linewidth=1.5)
        plt.axhline(-threshold, color='r', linestyle='dashed', linewidth=1.5)
        plt.title(f'Scatter Plot of Z-scores {col}')
        plt.xlabel('Index')
        plt.ylabel('Z-score')
        plt.grid(True)

        

        # Highlight outliers
        for i, (score, is_outlier) in enumerate(zip(z_scores[col], outliers)):
            if is_outlier:
                plt.text(i, score, f'{score:.2f}', fontsize=9, color='red', ha='left', va='bottom')

        plt.tight_layout()
        plt.show()

In [25]:


def IQR(df):
    summary = df.describe()
    Q1 = summary.loc['25%']
    Q3 = summary.loc['75%']
    IQR = Q3 - Q1

    lower_bounds = Q1 - 1.5 * IQR
    upper_bounds = Q3 + 1.5 * IQR
    outliers_dict = {}
    
    for feature in df.columns:
        lower_bound = lower_bounds[feature]
        upper_bound = upper_bounds[feature]
        
        # Identify outliers
        outliers = df[(df[feature] < lower_bound) | (df[feature] > upper_bound)]
        outliers_dict[feature] = outliers
        
    return outliers_dict, lower_bounds, upper_bounds

weather_iqr, weather_lower_bound, weather_upper_bound = IQR(weather_num[skewed_weather_columns])
generation_iqr, generation_lower_bound, generation_upper_bound = IQR(generation_num[skewed_energy_columns])



In [26]:
def visualIQR(data, lower_bound, upper_bound):
    for col in data.columns:
        # Plot Histogram of Z-scores
        plt.figure(figsize=(12, 5))

        # Histogram plot
        plt.subplot(1, 2, 1)
        plt.hist(data[col], bins=100, color='lightblue', edgecolor='black', alpha=0.7)
        plt.axvline(x=lower_bound[col], color='r', linestyle='--', label=f'Lower Bound ({lower_bound[col]})')
        plt.axvline(x=upper_bound[col], color='g', linestyle='--', label=f'Upper Bound ({upper_bound[col]})')

        # Customize plot
        plt.xlabel('Value')
        plt.ylabel('Frequency')
        plt.yscale('log')
        plt.title(f'Histogram with IQR Boundaries {col}')
        plt.legend()

        # Scatter Plot with Z-scores
        plt.subplot(1, 2, 2)
        plt.scatter(range(len(data[col])), data[col], label='Data Points', color='b')
        plt.axhline(y=lower_bound[col], color='r', linestyle='--', label=f'Lower Bound ({lower_bound[col]})')
        plt.axhline(y=upper_bound[col], color='g', linestyle='--', label=f'Upper Bound ({upper_bound[col]})')

        # Marking outliers
        outliers = data[(data[col] < lower_bound[col]) | (data[col] > upper_bound[col])]
        plt.scatter(np.where((data[col] < lower_bound[col]) | (data[col] > upper_bound[col]))[0], outliers[col], color='r', label='Outliers')

        # Customize plot
        plt.xlabel('Index')
        plt.ylabel('Value')
        plt.title(f'Scatter Plot with IQR Boundaries and Outliers {col}')
        plt.legend()
        


        plt.tight_layout()
        plt.show()

In [ ]:
visualIQR(data=weather_num[skewed_weather_columns], lower_bound=weather_lower_bound, upper_bound=weather_upper_bound)

In [ ]:
visualIQR(data=generation_num[skewed_energy_columns], lower_bound=generation_lower_bound, upper_bound=generation_upper_bound)

In [ ]:
visualZscore(z_scores=weather_zscores, outliers=weather_outliers)

In [ ]:
visualZscore(z_scores=generation_zscore, outliers=generation_outliers)

**Handling of outliers**

In [31]:
#weather dataset

cap_weather_columns=["temp", 'temp_min', 'temp_max', 'pressure', 'humidity', 'wind_speed', 'rain_1h', 'snow_3h']
remove_weather_columns=[]
transform_weather_columns=['rain_3h']

#cap rain at a different threshold 

#energy dataset

normal_energy_columns=['generation biomass', 'generation fossil hard coal', 'generation fossil oil',
                        'generation other', 'generation other renewable', 'total load forecast','total load actual',
                         'price day ahead', 'price actual']
skewed_energy_columns=['generation fossil brown coal/lignite', 'generation fossil gas','generation hydro pumped storage consumption',
                        'generation hydro run-of-river and poundage', 'generation hydro water reservoir', 'generation nuclear',
                         'generation solar', 'generation waste', 'generation wind onshore',
                           'forecast solar day ahead', 'forecast wind onshore day ahead']

cap_energy_columns=['generation fossil oil', 'generation nuclear', 'generation other renewable','generation waste', 'generation wind onshore', 'generation biomass']
#remove_energy_columns=['generation biomass']
transform_energy_columns=['generation fossil gas','generation hydro pumped storage consumption','generation hydro water reservoir' ]

In [ ]:
#capping
for col in weather_num[cap_weather_columns].columns:
    if col in skewed_weather_columns:
        if col == 'rain_1h':
            weather_num[col] = np.clip(weather_num[col], weather_lower_bound[col], 3)
        else:
            weather_num[col] = np.clip(weather_num[col], weather_lower_bound[col], weather_upper_bound[col])
    else:
        mean = np.mean(weather_num[col])
        std_dev = np.std(weather_num[col])
        lower_bound = mean - (3 * std_dev)
        upper_bound = mean + (3 * std_dev)
        weather_num[col] = np.clip(weather_num[col], lower_bound, upper_bound)

#log-transform
for col in weather_num[transform_weather_columns].columns:
    weather_num[col] = np.log1p(weather_num[col])

#capping
for col in generation_num[cap_energy_columns].columns:
    if col in skewed_energy_columns:
        generation_num[col] = np.clip(generation_num[col], generation_lower_bound[col], generation_upper_bound[col])
    else:
        mean = np.mean(generation_num[col])
        std_dev = np.std(generation_num[col])
        lower_bound = mean - (3 * std_dev)
        upper_bound = mean + (3 * std_dev)
        generation_num[col] = np.clip(generation_num[col], lower_bound, upper_bound)

#log-transform
for col in generation_num[transform_energy_columns].columns:
    generation_num[col] = np.log1p(generation_num[col])

#remove
#print(generation_outliers[remove_energy_columns])
"""true_indexes = generation_outliers[generation_outliers['generation biomass'] == True].index

print(true_indexes)

generation_num.drop(true_indexes, inplace=True)
data_generation.drop(true_indexes, inplace=True)
generation_num.reset_index(drop=True, inplace=True)
data_generation.reset_index(drop=True, inplace=True)"""

In [ ]:
visualData(data=weather_num, time=data_weather['dt_iso'])

In [ ]:
visualData(data=generation_num, time=data_generation['time'])

In [ ]:
df_encoded= pd.get_dummies(weather_cat, columns=['weather_main', 'weather_description', 'weather_icon',
       'weather_id'])
df_encoded=df_encoded.drop(columns=['city_name'])

df_encoded = df_encoded.astype(int)
df_encoded.head()


In [ ]:
weather_cat = pd.concat([weather_cat['city_name'], df_encoded], axis=1)
weather_cat.head()

In [37]:
#feature scaling
#weather dataset
norm_weather_columns=["temp", 'temp_min', 'temp_max', 'pressure', 'snow_3h']
skew_weather_columns=['humidity', 'wind_speed', 'rain_1h', 'wind_deg', 'rain_3h', 'clouds_all']
modal_weather_columns=[]

#cap rain at a different threshold 

#energy dataset


norm_energy_columns=['generation fossil gas', 'generation fossil oil', 'price day ahead', 'price actual']
#remove_energy_columns=['generation biomass']
skew_energy_columns=['generation fossil brown coal/lignite', 'generation hydro pumped storage consumption', 
                     'generation hydro run-of-river and poundage', 'generation hydro water reservoir', 'generation solar',
                     'generation waste', 'generation wind onshore', 'forecast solar day ahead', 'forecast wind onshore day ahead']
modal_energy_columns=['generation biomass', 'generation fossil hard coal', 'generation nuclear', 'generation other', 'generation other renewable',
                      'total load forecast','total load actual']

In [ ]:
from sklearn.preprocessing import StandardScaler, RobustScaler, QuantileTransformer
x = weather_num[norm_weather_columns].to_numpy()
standard_scaler = StandardScaler().fit(x)
weather_num[norm_weather_columns] = standard_scaler.transform(x)
x = generation_num[norm_energy_columns].to_numpy()
standard_scaler = StandardScaler().fit(x)
generation_num[norm_energy_columns] = standard_scaler.transform(x)

x = weather_num[skew_weather_columns].to_numpy()
robust_scaler = RobustScaler()
weather_num[skew_weather_columns] = robust_scaler.fit_transform(x)
x = generation_num[skew_energy_columns].to_numpy()
robust_scaler = RobustScaler()
generation_num[skew_energy_columns] = robust_scaler.fit_transform(x)

x = generation_num[modal_energy_columns].to_numpy()
quantile_transformer = QuantileTransformer(output_distribution="normal")
generation_num[modal_energy_columns]= quantile_transformer.fit_transform(x)

visualData(data=weather_num, time=data_weather['dt_iso'])
visualData(data=generation_num, time=data_generation['time'])

In [ ]:
combined_weather = pd.concat([data_weather['dt_iso'],weather_num, weather_cat], axis=1)
combined_weather.head()
combined_gen = pd.concat([data_generation['time'],generation_num], axis=1)
combined_gen.head()

In [ ]:
combined_weather.head()

In [ ]:
combined_weather['dt_iso'].iloc[0]

In [ ]:
timestamp_to_find = pd.Timestamp('2015-01-01 00:00:00+01:00', tz='UTC+01:00')
filtered_rows = combined_weather[combined_weather['dt_iso'] == timestamp_to_find]

# Print the filtered rows
print(filtered_rows)

In [ ]:
pivot_df = combined_weather.pivot_table(index='dt_iso', columns='city_name', aggfunc='first')

# Flatten multi-level columns (from city_name)
pivot_df.columns = [f"{col[0]}_{col[1]}" for col in pivot_df.columns]

# Reset index to make 'dt_iso' a column
pivot_df = pivot_df.reset_index()

# Print the reshaped DataFrame
pivot_df.head()

In [ ]:
pivot_df.shape

In [ ]:
combined_gen.shape

In [ ]:
merged_df = pd.merge(pivot_df, combined_gen, left_on='dt_iso', right_on='time', how='inner')
merged_df= merged_df.drop(columns=['time'])
# Print the merged dataframe
print(merged_df.shape)
merged_df.head()

In [ ]:
X = merged_df.drop('total load actual', axis=1)  # Features (predictors)
y = merged_df['total load actual']  # Target variable (what we want to predict)

"""# Perform the time-based split, setting shuffle=False to maintain order
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, shuffle=False
)"""
# First, split into training and temp (validation + test)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, shuffle=False)

# Now split the temp into validation and test sets (50% each of the remaining 30%)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, shuffle=False)

# Print the shape of the splits
print("Training set shape (X_train, y_train):", X_train.shape, y_train.shape)
print("Validation set shape (X_val, y_val):", X_val.shape, y_val.shape)
print("Testing set shape (X_test, y_test):", X_test.shape, y_test.shape)



In [48]:
X_train.to_csv('train/X_train.csv', index=False)
y_train.to_csv('train/y_train.csv', index=False)

X_val.to_csv('validation/X_val.csv', index=False)
y_val.to_csv('validation/y_val.csv', index=False)

X_test.to_csv('test/X_test.csv', index=False)
y_test.to_csv('test/y_test.csv', index=False)

In [ ]:
X = weather_num.drop('wind_deg', axis=1)  # Features
y = weather_num['wind_deg']  
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print("Training set shape:", X_train.shape, y_train.shape)
print("Testing set shape:", X_test.shape, y_test.shape)

In [ ]:
X.head()

In [ ]:
# PCA for dimensionality reduction
scaler = StandardScaler()
X = weather_num.drop('wind_deg', axis=1)  # Features
y = weather_num['wind_deg']  
X_scaled = scaler.fit_transform(X)
X_test_scaled = scaler.transform(X)
pca = PCA(n_components=len(X.columns))
X_pca = pca.fit_transform(X_scaled)


# Plot explained variance ratio
plt.figure(figsize=(12, 6))
plt.bar(range(1, len(pca.explained_variance_ratio_) + 1), pca.explained_variance_ratio_, alpha=0.7)
cumulative_variance = np.cumsum(pca.explained_variance_ratio_)
plt.plot(range(1, len(pca.explained_variance_ratio_) + 1), cumulative_variance, c='r', marker='o')

# Annotate cumulative variance on the plot
for i, value in enumerate(cumulative_variance):
    plt.text(i + 1, cumulative_variance[i], f'{value:.2f}', ha='center', va='bottom')

plt.title('Explained Variance Ratio by Components')
plt.xlabel('Principal Components')
plt.ylabel('Explained Variance Ratio')
plt.grid(True)
plt.legend(['Cumulative Explained Variance', 'Explained Variance Ratio'])
plt.show()

In [ ]:
# Print the difference in amount of features before and after PCA
print(f'Number of features before PCA: {X.shape[1]}')
print(f'Number of features after PCA: {X_pca.shape[1]}')

In [ ]:
# Save the preprocessed data to CSV files
merged_df.to_csv('preprocessed_data.csv', index=False)